In [1]:
# Calculate Mean Centered Rating (listens) vectors
# Calculate User Taste Profile Using Averaged Item Vectors Weighted by Mean Centered Ratings (listens)

In [2]:
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
af = pd.read_csv('../../Dropbox/datasets/audio_feats.csv')
trk = pd.read_csv('../../Dropbox/datasets/track_lookup.csv')
df=pd.read_csv('../../DSI-NYC-4/datasets/matrix.csv')

In [4]:
af['popularity']=af['popularity'].apply(lambda x: x/100.)
trk['popularity']=trk['popularity'].apply(lambda x: x/100.)
af['Artist_Track']=trk['Artist_Track']
af.sort('Artist_Track', inplace=True)
df['Total Plays']=df.T.sum()
df['Artist_Track']=df.index
af['Total Plays'] = df['Total Plays'].values
af=pd.merge(af.sort('Artist_Track'), df[['Total Plays', 'Artist_Track']], on=['Artist_Track','Total Plays'], how='inner')
 

In [5]:
def zero_cent(df, axis=0):
    df.replace(0,np.NaN)
    return df.apply(lambda x: x-x.mean(), axis = axis).replace(np.nan,0)

In [6]:
df = zero_cent(df.iloc[:,0:988])

In [7]:
knn = KNeighborsClassifier()
knn.fit(af.iloc[:,0:12], np.random.randint(2, size=24323))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [8]:
import pickle
with open('content_knn.pkl', 'w') as picklefile:
    pickle.dump(knn, picklefile)

In [9]:
import random as rm
def recommend_for_user(user, q=.75, pt=0, m=20):
    df_listened=df[df[user]>0]
    songs = df_listened[df_listened[user]>df_listened[user].quantile(q)][user]
    recs= []
    for s, p in songs.iteritems():
        n = p//songs.values.std()+1
        Artist_Track = s
        Neighbors = knn.kneighbors(af[af['Artist_Track'] == Artist_Track].iloc[:,0:12], n_neighbors=n, return_distance=False)
        for n in Neighbors[0][1:5]:
            try:
                recs.append(af[af['popularity']>pt][['Artist_Track','popularity']].loc[n])
            except:
                 pass
    rm.shuffle(recs)
    return recs

In [10]:
recommend_for_user('user_000577', pt=.2, m=20)

[Artist_Track    Dido: Look No Further
 popularity                       0.30
 Name: 5777, dtype: object, Artist_Track    José González: Killing For Love
 popularity                                 0.39
 Name: 10381, dtype: object, Artist_Track    The Magic Numbers: Long Legs
 popularity                              0.25
 Name: 21185, dtype: object, Artist_Track    Belle And Sebastian: If You Find Yourself Caug...
 popularity                                                   0.26
 Name: 2278, dtype: object, Artist_Track    Nat King Cole: The Christmas Song
 popularity                                   0.43
 Name: 14010, dtype: object, Artist_Track    The Decemberists: The Crane Wife 1 & 2
 popularity                                        0.33
 Name: 20364, dtype: object, Artist_Track    Pink Floyd: San Tropez
 popularity                        0.43
 Name: 15542, dtype: object, Artist_Track    M.I.A.: Paper Planes
 popularity                      0.66
 Name: 12094, dtype: object, Artis

In [11]:
df['user_000577'].sort_values(ascending=False).head(30)

Evanescence: My Immortal                           283.67
Rage Against The Machine: Beautiful World          228.67
Sia: Breathe Me                                    228.67
Massive Attack: Angel                              221.67
Damien Rice: 9 Crimes                              218.67
John Lennon: Imagine                               198.67
Iron & Wine: Such Great Heights                    178.67
Radiohead: Exit Music (For A Film)                 176.67
Lou Reed: Perfect Day                              169.67
The Dresden Dolls: Coin-Operated Boy               168.67
Gnarls Barkley: Crazy                              163.67
The Boomtown Rats: I Don'T Like Mondays            162.67
Damien Rice: The Blower'S Daughter                 159.67
Scissor Sisters: Return To Oz                      157.67
Jeff Buckley: Hallelujah                           155.67
Moby: God Moving Over The Face Of The Waters       153.67
Muse: Feeling Good                                 149.67
Michael Andrew

In [12]:
def recommend_by_track(track, n, pt=0.1, m=20):
    recs=[]
    Neighbors = knn.kneighbors(af[af['Artist_Track'] == track].iloc[:,0:12], n_neighbors=n, return_distance=False)
    for n in Neighbors[0][:]:
        try:
            recs.append(af[af['popularity']>pt][['Artist_Track','popularity']].loc[n])
        except:
             pass
    rm.shuffle(recs)
    return recs

In [13]:
recommend_by_track("Queen: Bohemian Rhapsody", n=100, pt=0)

[Artist_Track    Black Sabbath: Electric Funeral
 popularity                                 0.44
 Name: 2721, dtype: object, Artist_Track    Melanie C: Better Alone
 popularity                         0.16
 Name: 12839, dtype: object, Artist_Track    Coldplay: Fix You
 popularity                   0.77
 Name: 4494, dtype: object, Artist_Track    R.E.M.: Find The River
 popularity                        0.34
 Name: 16169, dtype: object, Artist_Track    Elbow: My Very Best
 popularity                     0.13
 Name: 6470, dtype: object, Artist_Track    Broken Social Scene: Cause = Time
 popularity                                   0.38
 Name: 3507, dtype: object, Artist_Track    Rhesus: Talk Talk Talk
 popularity                        0.01
 Name: 16674, dtype: object, Artist_Track    Spitalfield: In The Same Lifetime
 popularity                                   0.15
 Name: 18532, dtype: object, Artist_Track    Damien Rice: The Blower'S Daughter
 popularity                             

In [67]:
recommend_by_track("Queen: Bohemian Rhapsody", n=100, pt=0)

,Artist_Track,popularity
1048,Andrew Bird: Armchairs,0.29
7583,Frank Sinatra: My Way,0.59
4746,"Crosby, Stills, Nash & Young: Suite: Judy Blue...",0.59
6924,Europe: Carrie,0.60
20353,The Decemberists: Grace Cathedral Hill,0.31
2721,Black Sabbath: Electric Funeral,0.44
4482,Coldplay: A Message,0.60
2539,Billy Joel: New York State Of Mind,0.49
10776,Keane: On A Day Like Today,0.29
16075,Queen: Bohemian Rhapsody,0.71
